In [134]:
import os
import pandas as pd
import datetime

In [25]:
dd=tu.get_today_ticks('600031')
dd

,time,price,vol,type
0,091502,28.52,452,-
1,092038,28.51,3453,-
2,092050,28.50,3558,-
3,092059,28.48,3698,-
4,092108,28.47,3768,-
...,...,...,...,...
4772,145651,28.13,731,买入
4773,145654,28.15,145,卖出
4774,145657,28.15,448,卖出
4775,145701,28.14,170,买入


In [26]:
dd_clear=dd[dd['time']>='093030']
dd_clear

,time,price,vol,type
44,093032,28.00,2461,买入
45,093035,28.00,2828,买入
46,093038,28.00,1902,买入
47,093041,27.99,1642,买入
48,093044,27.98,1794,买入
...,...,...,...,...
4772,145651,28.13,731,买入
4773,145654,28.15,145,卖出
4774,145657,28.15,448,卖出
4775,145701,28.14,170,买入


In [27]:
dd_clear_buy=dd_clear[dd_clear['type']=='买入']
dd_clear_buy['total']=dd_clear_buy['price']*dd_clear_buy['vol']*100

<ipython-input-27-4f67fd5add16>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd_clear_buy['total']=dd_clear_buy['price']*dd_clear_buy['vol']*100


In [28]:
dd_clear_buy['total'].sum()

2836603986.0

In [29]:
dd_clear_buy['total'].sum()

2836603986.0

In [30]:
dd_clear_sale=dd_clear[dd_clear['type']=='卖出']
dd_clear_sale['total']=dd_clear_sale['price']*dd_clear_sale['vol']*100
dd_clear_sale['total'].sum()

<ipython-input-30-8332e6e35c5f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd_clear_sale['total']=dd_clear_sale['price']*dd_clear_sale['vol']*100


2501358046.0

In [31]:

print(dd_clear_buy['total'].sum()-dd_clear_sale['total'].sum())
print(dd_clear_buy['total'].sum()+dd_clear_sale['total'].sum())

335245940.0
5337962032.0


In [176]:
def analyze():
    param={'600031':70,'000333':28,'603185':7,'601995':34,'600737':200,'000876':200,'300869':52}
    for a,b,c in os.walk(r'F:\gu\202109\transform'):
        for i in c:
            file_name=i[0:6]
            month=int(i[6:8])
            day=int(i[8:10])
            print('开始处理：%s   %d月%d日的逐笔成交数据！' %(file_name,month,day))
            # print(file_name,'----',month,'----',day)
            file_str=os.path.join(a,i)
            # print(file_str)
            datas=pd.read_excel(file_str,sheet_name=0,names=['time','price','num','other','type'],parse_dates=[0])
            row,col=datas.shape
            # print(row,col)
            # print(datas)
            datas=datas[3:row-1]
            # print(datas)
            datas=datas[datas['time']>='09:30:00']
            # print(datas.shape)
            datas['total']=datas['price']*datas['num']
            buy=datas[datas['type']=='B']['total'].sum()
            buy_totall=round(buy/1000000,5)
            sale=datas[datas['type']=='S']['total'].sum()
            sale_totall=round(sale/1000000,5)
            total_add=buy_totall+sale_totall
            total_sub=buy_totall-sale_totall
            print('总买: %f \n总卖: %f\n总量: %f \n总差额: %f \n'  %(buy_totall,sale_totall,total_add,total_sub))
            #分析大单的情况:
            print('%s------%d' %(file_name,param[file_name]))
            big=datas[datas['num']>=param[file_name]]
            # print(big.shape)
            big_buy=big[big['type']=='B']
            big_sale=big[big['type']=='S']
            buy_total=big_buy['price']*big_buy['num']
            big_buy.insert(5,'SB_total',buy_total)
            sale_total=big_sale['price']*big_sale['num']
            big_sale.insert(5,'SB_total',sale_total)
            big_buy_total=big_buy['SB_total'].sum()/1000000
            big_sale_total=big_sale['SB_total'].sum()/1000000
            big_total_add=big_buy_total+big_sale_total
            big_total_sub=big_buy_total-big_sale_total
            #看看中间单的情况
            NaN=datas[datas['type'].isna()]
            # print(NaN)
            # NaN_total=NaN['price']*NaN['num']
            # NaN.insert(5,'total',NaN_total)
            NaN_total_sum=NaN['total'].sum()/1000000
            print('大买: %f \n大卖: %f\n大总量: %f \n大差额: %f \n'  %(big_buy_total,big_sale_total,big_total_add,big_total_sub))
            dat=pd.DataFrame([[datetime.date(2021,month,day),buy_totall,sale_totall,total_add,total_sub,big_buy_total,big_sale_total,big_total_add,big_total_sub,NaN_total_sum]],columns=['日期','总买','总卖','总量','总差额','大买','大卖','大总量','大差额','NaN'])
            # print(dat)
            # dat.to_excel(r'F:\gu\total\600031.xlsx')
            file_name=file_name+'.xlsx'
            file_path=os.path.join('F:\\gu\\total\\',file_name)
            # print(file_path)
            if(os.path.exists(file_path)):
                datas=pd.read_excel(file_path,sheet_name=0)
                datas=datas.append(dat)
                datas.to_excel(file_path,index=None)
            else:
                dat.to_excel(file_path,index=None)
            print('处理：%s   %d月%d日的逐笔成交数据完毕！！！' %(file_name,month,day))

In [177]:
analyze()

开始处理：000333   9月7日的逐笔成交数据！
总买: 12.312400 
总卖: 15.319210
总量: 27.631610 
总差额: -3.006810 

000333------28
大买: 11.693950 
大卖: 14.456834
大总量: 26.150784 
大差额: -2.762883 

           日期       总买        总卖        总量      总差额        大买         大卖  \
0  2021-09-07  12.3124  15.31921  27.63161 -3.00681  11.69395  14.456834   

         大总量       大差额      NaN  
0  26.150784 -2.762883  0.76169  
F:\gu\total\000333.xlsx
处理：000333.xlsx   9月7日的逐笔成交数据完毕！！！
开始处理：000333   9月8日的逐笔成交数据！
总买: 9.262030 
总卖: 9.616970
总量: 18.879000 
总差额: -0.354940 

000333------28
大买: 8.125090 
大卖: 8.723037
大总量: 16.848127 
大差额: -0.597947 

           日期       总买       总卖      总量      总差额       大买        大卖  \
0  2021-09-08  9.26203  9.61697  18.879 -0.35494  8.12509  8.723037   

         大总量       大差额     NaN  
0  16.848127 -0.597947  0.4622  
F:\gu\total\000333.xlsx
处理：000333.xlsx   9月8日的逐笔成交数据完毕！！！
开始处理：000333   9月9日的逐笔成交数据！
总买: 7.257860 
总卖: 8.353440
总量: 15.611300 
总差额: -1.095580 

000333------28
大买: 6.148481 
大卖: 7.201628


In [98]:
datas=pd.read_excel(r'F:\gu\20210903\xy.xlsx',sheet_name=0,names=['time','price','num','other','type'],parse_dates=[0])
row,col=datas.shape
datas=datas[3:row-1]
big=(datas[datas['num']>=70])
big_buy=big[big['type']=='S']
big_sale=big[big['type']=='B']
buy_total=big_buy['price']*big_buy['num']
big_buy.insert(5,'total',buy_total)
sale_total=big_sale['price']*big_sale['num']
big_sale.insert(5,'total',sale_total)
print('大买单： %s      \n大买单： %s' %(str(big_buy['total'].sum()/1000000) ,str(big_sale['total'].sum()/1000000)))
distance_volumn=(big_buy['total'].sum())-(big_sale['total'].sum())
print('大单的差量为：%f 亿'  %(distance_volumn/1000000))




大买单： 1.6483783099999991      
大买单： 1.8863967300000017
-0.23801842000000273
大单的差量为：-0.238018 亿


In [154]:
os.path.join("F:\\gu\\total\\","33.xls")
for a,b,c in os.walk(r'F:\gu\202109\transform'):
    for i in c:
        print(i)
        file_name=i[0:5]
        month=int(i[6:8])
        day=int(i[8:10])
        print(file_name,'----',month,'----',day)

0003330907.xlsx
00033 ---- 9 ---- 7
0003330908.xlsx
00033 ---- 9 ---- 8
0003330909.xlsx
00033 ---- 9 ---- 9
0008760907.xlsx
00087 ---- 9 ---- 7
3008690906.xlsx
30086 ---- 9 ---- 6
3008690907.xlsx
30086 ---- 9 ---- 7
3008690908.xlsx
30086 ---- 9 ---- 8
3008690909.xlsx
30086 ---- 9 ---- 9
6000310903.xlsx
60003 ---- 9 ---- 3
6000310906.xlsx
60003 ---- 9 ---- 6
6000310907.xlsx
60003 ---- 9 ---- 7
6000310908.xlsx
60003 ---- 9 ---- 8
6000310909.xlsx
60003 ---- 9 ---- 9
6007370903.xlsx
60073 ---- 9 ---- 3
6007370907.xlsx
60073 ---- 9 ---- 7
6007370908.xlsx
60073 ---- 9 ---- 8
6007370909.xlsx
60073 ---- 9 ---- 9
6019950906.xlsx
60199 ---- 9 ---- 6
6019950907.xlsx
60199 ---- 9 ---- 7
6019950908.xlsx
60199 ---- 9 ---- 8
6019950909.xlsx
60199 ---- 9 ---- 9
6031850909.xlsx
60318 ---- 9 ---- 9


In [110]:
temp=pd.read_excel(r'F:\gu\202109\transform\6000310909.xlsx',sheet_name=0,names=['time','price','num','other','type'],parse_dates=[0])
row,col=temp.shape
temp=temp[3:row-1]
temp_S=temp[temp['type']=='S']
temp_B=temp[temp['type']=='B']

In [113]:
temp_S['total']=temp_S['price']*temp_S['num']
temp_B['total']=temp_B['price']*temp_B['num']
temp_S_total=temp_S['total'].sum()/1000000
temp_B_total=temp_B['total'].sum()/1000000

<ipython-input-113-d2b77ad9e584>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_S['total']=temp_S['price']*temp_S['num']
<ipython-input-113-d2b77ad9e584>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_B['total']=temp_B['price']*temp_B['num']


In [171]:
not os.path.exists('F:\\gu\\total\\000333.xlsx')

True

In [155]:
param={'600031':70,'000333':28,'603185':7,'601995':34,'600737':200}

In [156]:
param['600031']

70

In [131]:
temp_null=temp[temp['type'].isna()]
temp_null

,time,price,num,other,type
3,09:25:00,28.09,8810,0,NaN
52,09:32:00,28.18,219,0,NaN
71,09:33:00,28.41,350,0,NaN
97,09:34:00,28.55,497,0,NaN
135,09:36:00,28.7,794,0,NaN
...,...,...,...,...,...
4248,14:32:00,28.29,357,0,NaN
4371,14:38:00,28.22,103,0,NaN
4407,14:40:00,28.25,135,0,NaN
4457,14:42:00,28.27,86,0,NaN


In [133]:
temp_null['total']=temp_null['price']*temp_null['num']
total=temp_null['total'].sum()/1000000
total

<ipython-input-133-54317fcaa8a8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_null['total']=temp_null['price']*temp_null['num']


1.36463498

In [71]:
import numpy as np
df = pd.DataFrame(np.random.randint(1,10,(4,5)),columns=["A","B","C","D","E"])
print(df)
df['F']=0
for i in range(df.shape[0]):
    df['F']=df['A']*df['B']
print(df)


   A  B  C  D  E
0  4  6  5  9  6
1  8  1  7  7  9
2  2  4  3  6  3
3  8  1  1  5  9
   A  B  C  D  E   F
0  4  6  5  9  6  24
1  8  1  7  7  9   8
2  2  4  3  6  3   8
3  8  1  1  5  9   8
